<a href="https://colab.research.google.com/github/vijayamurugan27/GenerativeAI/blob/main/GraphRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph database is better when you are doing 100 of documents. For a single document Vector databse is better.

# Modern AI Pro: Practitioner

In [1]:
!pip install --upgrade --quiet  langchain langchain-community  langchain-huggingface langchain-groq langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs sentence-transformers chromadb


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
os.environ['USER_AGENT'] = 'mitra_summarizer'
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## 1. Model Setup

In [4]:
import os
from google.colab import userdata
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
llm_groq = ChatGroq(model_name="llama3-70b-8192")

In [5]:
os.environ["NEO4J_URI"] = userdata.get("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = userdata.get("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = userdata.get("NEO4J_PASSWORD")

In [6]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Neo4jVector

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 2. Generate Vector Embeddings

In [8]:
loader = WebBaseLoader("https://www.cnn.com/2024/09/14/asia/india-nuclear-ballistic-missile-submarine-intl-hnk-ml/index.html")

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
len(docs)

69

In [9]:
db = Neo4jVector.from_documents(docs,embeddings,url=os.environ["NEO4J_URI"],username=os.environ["NEO4J_USERNAME"],password=os.environ["NEO4J_PASSWORD"],search_type='hybrid')

In [10]:
query = "Relationship between China and India"
docs_with_score = db.similarity_search_with_score(query, k=5)
# combine documents
results = [d[0].page_content for d in docs_with_score]
combined_docs = " ".join(results)
combined_docs

"The de facto border between India and China, known as the Line of Actual Control, has been a longtime flashpoint between the two. Troops most recently clashed there in 2022 and in 2020, when hand-to-hand fighting between the two sides resulted in the deaths of at least 20 Indian and four Chinese soldiers in Aksai Chin.\n    \n\n        India developing second-strike capabilities Another regional rival\n\n\n            It‚Äôs not just China that India is looking at with its sub development, according to Abhijit Singh, a senior fellow at the Observer Research Foundation in Mumbai.\n    \n\n\n\n\n\n\n\n\n\nPakistan's Foreign Secretary Muhammad Syrus Sajjad Qazi during an interview in Ankara, Turkey, on August 4, 2021.\n\nAli Balikci/Anadolu Agency/Getty Images\n\n\n\n\nRelated article\nPakistan¬†accuses India of extrajudicial killings on¬†its territory India and Pakistan have long been at odds in the disputed and heavily militarized region of Kashmir, which both countries claim in its en

In [11]:
prompt = "You are an advanced summarizing agent. Take the following context and answer the user query appropriately"
def vector_rag(query):
  docs_with_score = db.similarity_search_with_score(query, k=5)
  results = [d[0].page_content for d in docs_with_score]
  combined_docs = " ".join(results)

  rag_query = prompt + "The context is: " + combined_docs + "The question is: " + query
  return llm_groq.invoke(rag_query).content

In [12]:
vector_rag("tell me about india's nuclear submarine")

"According to the article, India has recently commissioned its second nuclear-capable ballistic missile submarine, called the Arighaat, which has joined its naval fleet. The Indian government has not released much information about the submarine's capabilities, but claims that it is significantly more advanced than its predecessor, which was commissioned eight years ago. The Arighaat is seen as a key part of India's efforts to establish a secure second-strike nuclear force, allowing it to hold targets in Pakistan and China at risk. The exact capabilities of the submarine remain unclear, as India has not released pictures or detailed information about it since its commissioning on August 29."

## Moving to graphs

In [13]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph()

In [14]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm_groq)
graph_documents = llm_transformer.convert_to_graph_documents(docs[50:55])
graph_documents

[GraphDocument(nodes=[Node(id='India', type='Country', properties={}), Node(id='China', type='Country', properties={}), Node(id='Pakistan', type='Country', properties={}), Node(id='Abhijit Singh', type='Person', properties={}), Node(id='Muhammad Syrus Sajjad Qazi', type='Person', properties={}), Node(id='Observer Research Foundation', type='Organization', properties={}), Node(id='Mumbai', type='Location', properties={}), Node(id='Ankara', type='Location', properties={}), Node(id='Turkey', type='Country', properties={})], relationships=[Relationship(source=Node(id='India', type='Country', properties={}), target=Node(id='China', type='Country', properties={}), type='RIVAL', properties={}), Relationship(source=Node(id='India', type='Country', properties={}), target=Node(id='Pakistan', type='Country', properties={}), type='RIVAL', properties={}), Relationship(source=Node(id='Abhijit Singh', type='Person', properties={}), target=Node(id='Observer Research Foundation', type='Organization', p

In [15]:
graph.add_graph_documents(
  graph_documents,
  baseEntityLabel=True,
  include_source=True
)

In [16]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional

class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(...,
        description="All the person, organization, or business entities that " "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages([(
            "system",
            "You are extracting organization and person entities from the text."),
            (
            "human",
            "Use the given format to extract information from the following"
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm_groq.with_structured_output(Entities)
entity_chain.invoke({"question": "who is the rival of India?"}).names

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


['who', 'India', 'rival']

In [18]:
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query,
            {limit:2})
            YIELD node,score
            CALL {
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS
              output
              UNION
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS
              output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

print(structured_retriever("Who is the rival of India?"))

India - RIVAL -> China
India - RIVAL -> Pakistan
India - COMPETITOR -> China
India - COMPETITOR -> Pakistan
India - DISPUTE -> Pakistan
India - CLAIM -> Kashmir
India - DEVELOPS -> Agni-V
India - JOINS -> Mirv Club
China - BACKER -> Pakistan
China - JOINS -> Mirv Club
Pakistan - ALLY -> China
Pakistan - CLAIM -> Kashmir
Pakistan - CLAIMS -> Mirv Technology
Abhijit Singh - MEMBEROF -> Observer Research Foundation
Muhammad Syrus Sajjad Qazi - REPRESENTATIVE -> Pakistan
New Delhi - CAPITAL -> India
Islamabad - CAPITAL -> Pakistan
Korda - MEMBER -> Federation Of American Scientists
Korda - EXPERT -> Multiple Independently Targetable Reentry Vehicles (Mirv)
Korda - HAS_CONCERN -> Proliferation Fears
Multiple Independently Targetable Reentry Vehicles (Mirv) - APPLIES_TO -> Land-Based Missiles
Submarines - CARRIES -> Multiple Independently Targetable Reentry Vehicles (Mirv)


In [19]:
def graph_retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    docs_with_score = db.similarity_search_with_score(question, k=7)
    results = [d[0].page_content for d in docs_with_score]
    combined_docs = " ".join(results)

    return f"""Structured data: {structured_data} Unstructured data: {combined_docs}"""

graph_retriever("Who is the rival of India?")

Search query: Who is the rival of India?


"Structured data: India - RIVAL -> China\nIndia - RIVAL -> Pakistan\nIndia - COMPETITOR -> China\nIndia - COMPETITOR -> Pakistan\nIndia - DISPUTE -> Pakistan\nIndia - CLAIM -> Kashmir\nIndia - DEVELOPS -> Agni-V\nIndia - JOINS -> Mirv Club\nChina - BACKER -> Pakistan\nChina - JOINS -> Mirv Club\nPakistan - ALLY -> China\nPakistan - CLAIM -> Kashmir\nPakistan - CLAIMS -> Mirv Technology\nAbhijit Singh - MEMBEROF -> Observer Research Foundation\nMuhammad Syrus Sajjad Qazi - REPRESENTATIVE -> Pakistan\nNew Delhi - CAPITAL -> India\nIslamabad - CAPITAL -> Pakistan\nKorda - MEMBER -> Federation Of American Scientists\nKorda - EXPERT -> Multiple Independently Targetable Reentry Vehicles (Mirv)\nKorda - HAS_CONCERN -> Proliferation Fears\nMultiple Independently Targetable Reentry Vehicles (Mirv) - APPLIES_TO -> Land-Based Missiles\nSubmarines - CARRIES -> Multiple Independently Targetable Reentry Vehicles (Mirv) Unstructured data: Another regional rival\n\n\n            It‚Äôs not just China 

In [20]:
def graph_rag(graph_user_query):
  rag_query = "You are a smart agent who will help answer a question based on the user context.The context is: " +\
                graph_retriever(graph_user_query) + "The question is: " + graph_user_query
  return llm_groq.invoke(rag_query).content


print(graph_rag("Who is the rival of India?"))

Search query: Who is the rival of India?


Based on the structured data, India has two rivals: China and Pakistan.


In [21]:
print(graph_rag("what are india's key missiles"))

Search query: what are india's key missiles


Based on the provided structured and unstructured data, India's key missiles are:

1. **Agni-V**: A long-range nuclear-capable intercontinental ballistic missile with a range of over 5000 kilometers, capable of carrying a 1000 kg nuclear warhead, and has three rocket motors.

Note: The Agni-V missile has enabled India to join the select club of nations like the United States, UK, Russia, France, and China that have the capability to operate missiles across continents.


In [22]:
club = graph_rag("what club is India a member of?")

Search query: what club is India a member of?


In [23]:
club

'Based on the structured data, I can answer that India is a member of the "Mirv Club". This is stated in the fact "India - JOINS -> Mirv Club". Additionally, the unstructured data also confirms this, mentioning that "India announced to great fanfare in April that it had joined the MIRV club...".'